In [25]:
## this is for masking version of the bert model!
"""

# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

"""

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
## for embedding!!, use BertModel and not AutoModelForMaskedLM

from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\bert\bert-base-uncased-venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
text = ["abc"]

# Tokenize and encode text using batch_encode_plus
# The function returns a dictionary containing the token IDs and attention masks
encoding = tokenizer.batch_encode_plus(
    text,                    # List of input texts
    padding=True,              # Pad to the maximum sequence length
    truncation=True,           # Truncate to the maximum sequence length if necessary
    return_tensors='pt',      # Return PyTorch tensors
    add_special_tokens=True    # Add special tokens CLS and SEP
)

print(encoding) 

input_ids = encoding['input_ids']  # Token IDs
# print input IDs
print(f"Input ID: {input_ids}")
attention_mask = encoding['attention_mask']  # Attention mask
# print attention mask
print(f"Attention mask: {attention_mask}")


# Generate embeddings using BERT model
with torch.no_grad(): ## need import torch if want use/do this
    outputs = model(input_ids, attention_mask=attention_mask)
    word_embeddings = outputs.last_hidden_state  # This contains the embeddings

# Decode the token IDs back to text
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
#print decoded text
print(f"Decoded Text: {decoded_text}")

In [2]:
text = "abc123"

# Tokenize the text again for reference
tokenized_text = tokenizer.tokenize(text)
#print tokenized text
print(f"tokenized Text: {tokenized_text}")
# Encode the text
encoded_text = tokenizer.encode(text, return_tensors='pt')  # Returns a tensor
# Print encoded text
print(f"Encoded Text: {encoded_text}")

encoded_text_plus = tokenizer.encode_plus(text, return_tensors='pt')
print(encoded_text_plus)

tokenized Text: ['abc', '##12', '##3']
Encoded Text: tensor([[  101,  5925, 12521,  2509,   102]])
{'input_ids': tensor([[  101,  5925, 12521,  2509,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


In [4]:
output = model(encoded_text)
print("#1: ", output)

#1:  MaskedLMOutput(loss=None, logits=tensor([[[ -6.5954,  -6.5422,  -6.5436,  ...,  -5.7315,  -5.6304,  -3.7748],
         [ -7.5854,  -7.7376,  -7.7166,  ...,  -7.6155,  -7.8209,  -2.3627],
         [ -5.1827,  -5.8171,  -5.9971,  ...,  -5.1309,  -6.4651,  -3.5165],
         [-10.3041, -10.6987, -10.7929,  ...,  -9.5141,  -9.8526,  -6.2101],
         [-11.1503, -10.8655, -10.8055,  ...,  -8.7106,  -9.5394,  -8.8477]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [21]:
output = model(**encoded_text) ## cannot cos "argument after ** must be a mapping, not Tensor"
print("#2: ", output)

TypeError: BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [6]:
output = model(encoded_text_plus)
print("#3: ", output)

TypeError: list indices must be integers or slices, not tuple

In [7]:
output = model(**encoded_text_plus)
print("#4: ", output)

#4:  MaskedLMOutput(loss=None, logits=tensor([[[ -6.5954,  -6.5422,  -6.5436,  ...,  -5.7315,  -5.6304,  -3.7748],
         [ -7.5854,  -7.7376,  -7.7166,  ...,  -7.6155,  -7.8209,  -2.3627],
         [ -5.1827,  -5.8171,  -5.9971,  ...,  -5.1309,  -6.4651,  -3.5165],
         [-10.3041, -10.6987, -10.7929,  ...,  -9.5141,  -9.8526,  -6.2101],
         [-11.1503, -10.8655, -10.8055,  ...,  -8.7106,  -9.5394,  -8.8477]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [3]:
def get_word_embedding(text):
    encoded_text_with_input_ids_and_attention_masks_etc = tokenizer.encode_plus(text, return_tensors="pt")
    model_output =  model(**encoded_text_with_input_ids_and_attention_masks_etc)
    """
    print(tokenizer.tokenize(text))
    print(encoded_text_with_input_ids_and_attention_masks_etc)
    
    print(model_output)

    print(model_output[0])
    print(model_output.last_hidden_state)
    """
    return model_output[0]
    

In [4]:
get_word_embedding("i want to kill people")

NameError: name 'tokenizer' is not defined

In [16]:
def get_word_embedding(text):
    encoded_text_with_input_ids_and_attention_masks_etc = tokenizer.encode_plus(text, return_tensors="pt")
    model_output =  model(**encoded_text_with_input_ids_and_attention_masks_etc)
    return model_output[0][0]

In [17]:
print(get_word_embedding("i love you"))

tensor([[-0.0106,  0.1323,  0.1282,  ..., -0.4946,  0.0366,  0.1418],
        [ 0.2336,  0.2490, -0.1087,  ..., -0.3529,  0.3312,  0.3367],
        [ 0.9315,  0.9158,  1.1255,  ..., -0.9469, -0.0023,  0.0410],
        [-0.1725, -0.7705,  0.8230,  ..., -0.2080, -0.0336, -0.7374],
        [ 0.6461, -0.0204, -0.1173,  ..., -0.2163, -0.6109, -0.3366]],
       grad_fn=<SelectBackward0>)


In [4]:
def get_word_embedding_w_token(text):
    encoded_text_with_input_ids_and_attention_masks_etc = tokenizer.encode_plus(text, return_tensors="pt")
    model_output =  model(**encoded_text_with_input_ids_and_attention_masks_etc)
    input_ids = encoded_text_with_input_ids_and_attention_masks_etc["input_ids"]
    #print(input_ids)
    #print(tokenizer.decode(input_ids[0])) ## can "skip_special_tokens=True" if want as arg
    #print(tokenizer.convert_ids_to_tokens(input_ids[0]))
    tokenized_text_with_special_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    return list(zip(tokenized_text_with_special_tokens, model_output[0][0]))
get_word_embedding_w_token("hi adasd")

[('[CLS]',
  tensor([-3.5563e-01,  7.6927e-02, -1.0376e-01, -4.0746e-02, -2.1151e-01,
           8.2666e-02,  2.2463e-01,  2.4771e-01, -2.0665e-01, -4.6616e-02,
          -9.9115e-02, -1.7627e-02, -6.3732e-02,  2.5278e-01,  9.8815e-03,
           9.7530e-02, -2.6173e-01,  3.9854e-01,  2.9848e-01, -1.4321e-01,
          -2.3226e-01, -1.8624e-01, -2.2774e-01, -3.6241e-01,  4.7075e-02,
          -4.9511e-02, -4.5702e-02, -7.3345e-02,  2.2849e-01,  2.2118e-01,
          -8.9468e-03,  2.4675e-01, -1.1940e-01,  5.2065e-03,  1.8890e-02,
           1.2755e-01, -1.0527e-02, -1.9132e-02, -1.1296e-02,  2.1618e-01,
           4.9049e-02, -8.2607e-02,  1.5517e-01, -4.3656e-02, -1.1945e-02,
          -2.3611e-01, -1.7448e+00,  4.5237e-02, -2.5230e-01, -7.9770e-04,
          -2.5913e-01,  4.7024e-02,  1.4371e-01,  1.8214e-01, -6.4814e-02,
           4.5649e-01, -1.2924e-01,  5.4144e-01,  1.6165e-01,  2.0138e-01,
          -1.8698e-01, -1.5207e-01, -1.1585e-01,  1.5808e-01, -2.8148e-04,
           1.4

In [5]:
import torch

In [6]:
def show_tok_word_emb(some_text, cut_ten=False):
    for tok, emb in get_word_embedding_w_token(some_text):
        if cut_ten:
            print(f"{tok}: {emb[:10]}")
        else:
            print(f"{tok}: {emb}")
show_tok_word_emb("hi there basaew", True)

[CLS]: tensor([-0.0822,  0.3846, -0.1603, -0.1345, -0.4700, -0.1406,  0.2869,  0.3630,
         0.1515, -0.1577], grad_fn=<SliceBackward0>)
hi: tensor([ 0.1907,  0.7592,  0.7074, -0.8120, -0.5279,  0.3422, -0.2485, -0.2750,
         0.0132, -0.6560], grad_fn=<SliceBackward0>)
there: tensor([-0.7504,  0.7193, -0.3853, -0.0587,  0.3199,  0.0936, -0.2988,  1.1447,
         0.4214, -0.4911], grad_fn=<SliceBackward0>)
bas: tensor([ 0.4593,  0.4747, -0.0241, -0.5151, -0.8363, -0.6710,  0.7688, -0.1853,
        -0.5306, -0.4110], grad_fn=<SliceBackward0>)
##ae: tensor([ 0.1660,  0.2491,  0.5833, -0.5256,  0.0690,  0.1758,  0.2441, -0.0761,
        -0.0694,  0.3108], grad_fn=<SliceBackward0>)
##w: tensor([ 0.2123,  0.1483,  0.0374, -0.1016, -0.1094, -0.0771,  0.1752,  0.1314,
        -0.0956, -0.1034], grad_fn=<SliceBackward0>)
[SEP]: tensor([ 0.7705,  0.0766, -0.3977,  0.4511, -0.4394, -0.6858,  0.3326, -0.8821,
         0.6056, -0.0393], grad_fn=<SliceBackward0>)


In [7]:
show_tok_word_emb("slap you", True)

[CLS]: tensor([-0.2264,  0.4792, -0.2430,  0.0325, -0.4012, -0.6995,  0.2460,  0.9113,
        -0.2310, -0.1256], grad_fn=<SliceBackward0>)
slap: tensor([ 0.2286,  0.6143,  0.4380, -0.5413, -0.0462, -0.0814,  0.1635, -0.0780,
        -0.3251, -0.5345], grad_fn=<SliceBackward0>)
you: tensor([-1.2533,  0.2097, -0.1444, -0.8390, -0.8554,  0.2319,  0.0104,  0.9878,
        -1.1859, -0.1656], grad_fn=<SliceBackward0>)
[SEP]: tensor([ 0.7715,  0.0902, -0.2625,  0.4693, -0.4473, -0.5719,  0.6055, -0.5190,
         0.6415,  0.3206], grad_fn=<SliceBackward0>)


In [8]:
show_tok_word_emb("you slap", True)

[CLS]: tensor([-0.3022,  0.3190, -0.6128,  0.1541, -0.3523, -0.4283,  0.1539,  0.2861,
        -0.1903, -0.1779], grad_fn=<SliceBackward0>)
you: tensor([-0.2600,  0.4986, -0.1865, -0.4177, -0.7537,  0.0725, -0.3126,  0.6242,
        -0.9493,  0.0482], grad_fn=<SliceBackward0>)
slap: tensor([-0.0172,  0.2409, -0.6623, -0.5686, -0.6019, -0.1607,  0.7887,  0.4799,
        -0.5663, -0.4142], grad_fn=<SliceBackward0>)
[SEP]: tensor([ 0.9696,  0.0754, -0.1899,  0.5092, -0.3665, -0.5967,  0.5519, -0.5004,
         0.6059,  0.2141], grad_fn=<SliceBackward0>)


In [1]:
def get_cls_embedding(text): # sort of like sentence embedding maybe? i hope so
    return get_word_embedding(text)[:][0]

In [18]:
get_word_embedding("you slap")

tensor([[-0.3022,  0.3190, -0.6128,  ..., -0.1108,  0.2266,  0.3345],
        [-0.2600,  0.4986, -0.1865,  ...,  0.2049,  0.8535, -0.6136],
        [-0.0172,  0.2409, -0.6623,  ...,  0.2108,  0.3582, -0.3635],
        [ 0.9696,  0.0754, -0.1899,  ..., -0.0337, -0.8699, -0.3466]],
       grad_fn=<SelectBackward0>)

In [19]:
get_cls_embedding("you slap")

tensor([-3.0216e-01,  3.1903e-01, -6.1283e-01,  1.5407e-01, -3.5226e-01,
        -4.2829e-01,  1.5390e-01,  2.8611e-01, -1.9028e-01, -1.7794e-01,
         6.2591e-02,  2.2631e-01, -2.7719e-01,  4.4759e-01,  8.0433e-04,
         1.1076e-01, -9.8800e-02,  2.7483e-01,  1.9675e-02, -1.0155e-01,
         1.9854e-01,  3.4159e-02, -7.7434e-02,  1.0788e-01, -1.1583e-02,
        -1.2412e-02, -1.8740e-01, -4.4906e-01,  9.0015e-02, -5.2500e-02,
        -1.2867e-01,  1.1498e-02, -4.2578e-01, -3.0835e-02, -1.1381e-01,
        -6.6668e-02,  8.3218e-02,  3.4710e-01, -2.2852e-01, -2.0305e-01,
        -1.2648e-01,  2.4238e-01, -1.7087e-01,  1.6629e-02,  4.3587e-02,
         1.4640e-02, -1.9185e+00,  1.5873e-01,  5.6473e-02, -5.6386e-01,
        -5.0768e-02,  1.6419e-01,  3.1613e-01, -7.6420e-02,  2.7087e-01,
         2.8472e-01, -3.1631e-05,  6.9438e-01, -2.7928e-01,  7.0762e-02,
         2.9739e-02,  2.1687e-01, -4.5777e-02,  2.3344e-02,  2.2826e-01,
         2.1865e-01,  9.3328e-02,  7.7339e-01, -4.1

In [20]:
get_word_embedding("you slap")[0], get_word_embedding("you slap")[:][0]

(tensor([-3.0216e-01,  3.1903e-01, -6.1283e-01,  1.5407e-01, -3.5226e-01,
         -4.2829e-01,  1.5390e-01,  2.8611e-01, -1.9028e-01, -1.7794e-01,
          6.2591e-02,  2.2631e-01, -2.7719e-01,  4.4759e-01,  8.0433e-04,
          1.1076e-01, -9.8800e-02,  2.7483e-01,  1.9675e-02, -1.0155e-01,
          1.9854e-01,  3.4159e-02, -7.7434e-02,  1.0788e-01, -1.1583e-02,
         -1.2412e-02, -1.8740e-01, -4.4906e-01,  9.0015e-02, -5.2500e-02,
         -1.2867e-01,  1.1498e-02, -4.2578e-01, -3.0835e-02, -1.1381e-01,
         -6.6668e-02,  8.3218e-02,  3.4710e-01, -2.2852e-01, -2.0305e-01,
         -1.2648e-01,  2.4238e-01, -1.7087e-01,  1.6629e-02,  4.3587e-02,
          1.4640e-02, -1.9185e+00,  1.5873e-01,  5.6473e-02, -5.6386e-01,
         -5.0768e-02,  1.6419e-01,  3.1613e-01, -7.6420e-02,  2.7087e-01,
          2.8472e-01, -3.1631e-05,  6.9438e-01, -2.7928e-01,  7.0762e-02,
          2.9739e-02,  2.1687e-01, -4.5777e-02,  2.3344e-02,  2.2826e-01,
          2.1865e-01,  9.3328e-02,  7.

In [11]:
import torch
import math
import numpy as np
def get_length(embedding_1d):
    sum = 0
    for i in embedding_1d:
        sum+=(i**2)
    return math.sqrt(sum)
def normalise_embedding(embedding_1d):
    length = get_length(embedding_1d)
    for i in range(len(embedding_1d)):
        embedding_1d[i] /= length
def get_normalise_embedding(embedding_1d):
    if type(embedding_1d) is torch.Tensor:
        temp_embedding_1d = (embedding_1d.detach().numpy()).copy()
    else:
        temp_embedding_1d = embedding_1d.copy()
    length = get_length(temp_embedding_1d)
    for i in range(len(temp_embedding_1d)):
        temp_embedding_1d[i] /= length
    return temp_embedding_1d


def cosine_sim(embedding_1, embedding_2):
    embedding_1 = get_normalise_embedding(embedding_1)
    embedding_2 = get_normalise_embedding(embedding_2)
    sim_sum = 0
    for e_1, e_2 in zip(embedding_1, embedding_2):
        sim_sum += (e_1*e_2)
    return sim_sum
def norm_ed_cosine_sim(embedding_1, embedding_2):
    sim_sum = 0
    for e_1, e_2 in zip(embedding_1, embedding_2):
        sim_sum += (e_1*e_2)
    return sim_sum

In [14]:
aa = get_normalise_embedding(get_cls_embedding("hi there"))
print(get_length(aa))

0.9999999959719127


In [21]:
cosine_sim(get_cls_embedding("i love you"), get_cls_embedding(" i like you"))

0.9716051474177263

In [22]:
cosine_sim(get_cls_embedding("i love you"), get_cls_embedding(" i hate you"))

0.9689928427400734

In [23]:
cosine_sim(get_cls_embedding("i love you"), get_cls_embedding(" i am dying and would like to be saved so i can kill you"))

0.8936620383051874

In [24]:
cosine_sim(get_cls_embedding("employee benefit"), get_cls_embedding("tax relief"))

0.9317314271919477

In [25]:
cosine_sim(get_cls_embedding("employee benefit"), get_cls_embedding("salary bonus"))

0.8601194937180958

##### 2 different ones lol, but no "s" is what normally use bah, the "s" one use for within the naively avg and all

In [ ]:
def get_word_embedding(text):
    encoded_text_with_input_ids_and_attention_masks_etc = tokenizer.encode_plus(text, return_tensors="pt")
    model_output =  model(**encoded_text_with_input_ids_and_attention_masks_etc)
    return model_output[0][0]

def get_word_embeddings(sentence):
    t_encoded_input = tokenizer(sentence, return_tensors='pt'
                             , padding=True, truncation=True, add_special_tokens=True ## idk if need??? got huge diff?
                             )
    t_output = model(**t_encoded_input)    
    return t_output.last_hidden_state

#### below is like other laptop try attempts on stuff like avg embedding

In [20]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

def get_word_embeddings(sentence):
    t_encoded_input = tokenizer(sentence, return_tensors='pt'
                             , padding=True, truncation=True, add_special_tokens=True ## idk if need??? got huge diff?
                             )

    #print(tokenizer.tokenize(sentence))

    #print("input_id:", t_encoded_input["input_ids"])
    #print("attention_mask:", t_encoded_input["attention_mask"])

    t_output = model(**t_encoded_input)
    #t_output.last_hidden_state.shape, t_output.last_hidden_state
    
    return t_output.last_hidden_state

def text_embedding_via_avg_naively(text):
    word_embedding_list = get_word_embeddings(text)[0]
    token_count = len(word_embedding_list)
    sum_tensor = torch.tensor(word_embedding_list[0])
    for word_emb in word_embedding_list[1:]:
        sum_tensor += word_emb
    avg_tensor = sum_tensor/token_count
    return avg_tensor
def text_embedding_via_avg_naively(text): ## never normalise each word embedding prior... is it necessary? like stronger words affect more... or dw?
    word_embedding_list = get_word_embeddings(text)[0]
    token_count = len(word_embedding_list)
    sum_tensor = word_embedding_list[0].clone().detach().requires_grad_(False)
    for word_emb in word_embedding_list[1:]:
        sum_tensor += word_emb
    avg_tensor = sum_tensor/token_count
    return avg_tensor


def text_embedding_via_avg_naively_normlised_words(text): ## never normalise each word embedding prior... is it necessary? like stronger words affect more... or dw?
    word_embedding_list = [get_normalise_embedding(word_emb) for word_emb in get_word_embeddings(text)[0]]
    token_count = len(word_embedding_list)
    sum_tensor = word_embedding_list[0]
    for word_emb in word_embedding_list[1:]:
        sum_tensor += word_emb
    avg_tensor = sum_tensor/token_count
    return avg_tensor


In [34]:
def naively_avg_cosine_sim(t1, t2):
    return cosine_sim(text_embedding_via_avg_naively(t1), text_embedding_via_avg_naively(t2))
    #return norm_ed_cosine_sim(get_normalise_embedding(text_embedding_via_avg_naively(t1)), get_normalise_embedding(text_embedding_via_avg_naively(t2)))
def naively_normalised_avg_cosine_sim(t1, t2):
    return cosine_sim(text_embedding_via_avg_naively_normlised_words(t1), text_embedding_via_avg_naively_normlised_words(t2))
    #return norm_ed_cosine_sim(get_normalise_embedding(text_embedding_via_avg_naively_normlised_words(t1)), get_normalise_embedding(text_embedding_via_avg_naively_normlised_words(t2)))

In [31]:
naively_avg_cosine_sim("i love you", "i like you")

0.7771767107832801

In [35]:
naively_normalised_avg_cosine_sim("i love you", "i like you")

0.7792556218753361

In [36]:
naively_avg_cosine_sim("i love you", "i hate you")

0.818678379633063

In [37]:
naively_normalised_avg_cosine_sim("i love you", "i hate you")

0.8233166842416892

In [38]:
naively_avg_cosine_sim("i love you", "food is the best way to relieve hunger")

0.522591160227222

In [39]:
naively_avg_cosine_sim("i love food", "food is the best way to relieve hunger")

0.6090393860771925

In [40]:
naively_normalised_avg_cosine_sim("i love you", "food is the best way to relieve hunger")

0.5243620184164968

In [41]:
naively_normalised_avg_cosine_sim("i love food", "food is the best way to relieve hunger")

0.617445310014034

In [47]:
naively_normalised_avg_cosine_sim("i love trash that can be pooped and vomiting shit which cannot be eaten", "food is the best way to relieve hunger and i love food")

0.74544929323595

In [40]:
get_word_embedding("hi there")[:1]

tensor([[-1.4789e-01,  3.9649e-01, -2.4775e-01, -2.0591e-01, -3.3936e-01,
         -4.9942e-01,  5.7701e-01,  6.1992e-01,  1.1338e-01, -1.7785e-01,
          2.1542e-01, -9.2569e-02,  1.1369e-01,  6.0125e-01,  1.2243e-01,
         -1.4392e-01, -1.4674e-01,  3.5705e-01,  2.4601e-01,  2.6711e-02,
          1.9500e-01, -3.0176e-01,  7.4834e-02,  5.2450e-02, -2.5367e-01,
         -2.6850e-01, -1.7908e-01, -3.0084e-01,  2.6835e-01, -1.0506e-01,
         -1.4552e-02,  3.0471e-01, -4.3338e-01, -1.3430e-01,  1.6945e-01,
          2.1488e-01,  1.3354e-01,  1.5895e-01, -3.3153e-01,  9.7612e-02,
         -1.0803e-01,  4.5685e-02,  1.6506e-01,  1.7767e-01, -1.2236e-01,
         -4.3633e-01, -2.7322e+00, -1.5028e-01, -2.2578e-01, -5.7526e-01,
          5.2121e-01, -3.9550e-01,  1.1658e-01, -1.3485e-01, -1.2878e-01,
          4.6183e-01, -4.6208e-01,  4.1482e-01, -3.1442e-02,  2.8678e-01,
          6.9310e-04, -1.0354e-01,  8.6396e-03,  3.3226e-01,  5.5989e-02,
          1.8962e-01,  2.6571e-01,  5.

In [29]:
get_cls_embedding("hi there")

tensor([-1.4789e-01,  3.9649e-01, -2.4775e-01, -2.0591e-01, -3.3936e-01,
        -4.9942e-01,  5.7701e-01,  6.1992e-01,  1.1338e-01, -1.7785e-01,
         2.1542e-01, -9.2569e-02,  1.1369e-01,  6.0125e-01,  1.2243e-01,
        -1.4392e-01, -1.4674e-01,  3.5705e-01,  2.4601e-01,  2.6711e-02,
         1.9500e-01, -3.0176e-01,  7.4834e-02,  5.2450e-02, -2.5367e-01,
        -2.6850e-01, -1.7908e-01, -3.0084e-01,  2.6835e-01, -1.0506e-01,
        -1.4552e-02,  3.0471e-01, -4.3338e-01, -1.3430e-01,  1.6945e-01,
         2.1488e-01,  1.3354e-01,  1.5895e-01, -3.3153e-01,  9.7612e-02,
        -1.0803e-01,  4.5685e-02,  1.6506e-01,  1.7767e-01, -1.2236e-01,
        -4.3633e-01, -2.7322e+00, -1.5028e-01, -2.2578e-01, -5.7526e-01,
         5.2121e-01, -3.9550e-01,  1.1658e-01, -1.3485e-01, -1.2878e-01,
         4.6183e-01, -4.6208e-01,  4.1482e-01, -3.1442e-02,  2.8678e-01,
         6.9310e-04, -1.0354e-01,  8.6396e-03,  3.3226e-01,  5.5989e-02,
         1.8962e-01,  2.6571e-01,  5.6467e-01, -4.2

In [41]:
def get_sentence_embedding(sentence):
    #sentence_embeddings = torch.nn.functional.normalize(torch.tensor(get_cls_embedding(sentence)), p=2, dim=1) ## the "torch.nn.functional.normalize" function needs a tensor with like tensor(s) in tensor, so cannot just 1d tensor??? idk
    sentence_embeddings = torch.nn.functional.normalize(torch.tensor(get_word_embedding(sentence)[:1]), p=2, dim=1)
    return sentence_embeddings

In [42]:
def sent_cos_sim(t1, t2):
    return cosine_sim(get_sentence_embedding(t1)[0], get_sentence_embedding(t2)[0])

In [44]:
get_sentence_embedding("hi there")

C:\Users\Audric Ho\AppData\Local\Temp\ipykernel_10296\1698119748.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentence_embeddings = torch.nn.functional.normalize(torch.tensor(get_word_embedding(sentence)[:1]), p=2, dim=1)


tensor([[-9.9402e-03,  2.6649e-02, -1.6652e-02, -1.3840e-02, -2.2809e-02,
         -3.3567e-02,  3.8782e-02,  4.1666e-02,  7.6206e-03, -1.1954e-02,
          1.4478e-02, -6.2217e-03,  7.6412e-03,  4.0411e-02,  8.2289e-03,
         -9.6733e-03, -9.8628e-03,  2.3998e-02,  1.6535e-02,  1.7953e-03,
          1.3106e-02, -2.0282e-02,  5.0297e-03,  3.5252e-03, -1.7050e-02,
         -1.8046e-02, -1.2036e-02, -2.0220e-02,  1.8036e-02, -7.0615e-03,
         -9.7804e-04,  2.0480e-02, -2.9128e-02, -9.0266e-03,  1.1389e-02,
          1.4442e-02,  8.9755e-03,  1.0684e-02, -2.2283e-02,  6.5606e-03,
         -7.2608e-03,  3.0706e-03,  1.1094e-02,  1.1941e-02, -8.2243e-03,
         -2.9326e-02, -1.8363e-01, -1.0101e-02, -1.5175e-02, -3.8664e-02,
          3.5031e-02, -2.6582e-02,  7.8358e-03, -9.0633e-03, -8.6557e-03,
          3.1040e-02, -3.1057e-02,  2.7881e-02, -2.1132e-03,  1.9275e-02,
          4.6584e-05, -6.9592e-03,  5.8068e-04,  2.2331e-02,  3.7631e-03,
          1.2745e-02,  1.7859e-02,  3.

In [45]:
sent_cos_sim("tax relief", "employee benefit")

C:\Users\Audric Ho\AppData\Local\Temp\ipykernel_10296\1698119748.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentence_embeddings = torch.nn.functional.normalize(torch.tensor(get_word_embedding(sentence)[:1]), p=2, dim=1)


0.9317314202044775

In [46]:
sent_cos_sim("the world is green", "i hate soda")

C:\Users\Audric Ho\AppData\Local\Temp\ipykernel_10296\1698119748.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentence_embeddings = torch.nn.functional.normalize(torch.tensor(get_word_embedding(sentence)[:1]), p=2, dim=1)


0.9344516149574389

In [50]:
get_length(get_sentence_embedding("i love rice")[0])

C:\Users\Audric Ho\AppData\Local\Temp\ipykernel_10296\1698119748.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentence_embeddings = torch.nn.functional.normalize(torch.tensor(get_word_embedding(sentence)[:1]), p=2, dim=1)


0.9999997615813925

In [51]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False